Fine tuning eduge dataset for news classification.
BERT 32K 512

In [0]:
MODEL            = 'model-32k'
MODEL_CHECKPOINT = 'model.ckpt-4000000'
MODEL_BUCKET     = 'gs://mongolian-bert-models/model-32k-512-4000000'

#if False:
#  # for non public GCloud bucket
#  from google.colab import auth
#  auth.authenticate_user()

#import getpass
#GITHUB_USERNAME = getpass.getpass(prompt='Хэн бээ   : ')
#GITHUB_PASSWORD = getpass.getpass(prompt='Нууц үгээ : ')

In [0]:
import os
from os.path import exists, join, basename, splitext

is_on_colab = True
project_path = 'mongolian-bert'
try:
  import colab
except ModuleNotFoundError:
  is_on_colab = False
  project_path = '../../mongolian-bert'
  
import sys
sys.path.append(project_path)

In [3]:
if is_on_colab:
  # we are on Colab, clone our project
  if not exists(project_path):
    print("checking out")
    !git clone -q --recursive https://github.com/tugstugi/mongolian-bert.git

checking out


In [4]:
!pip install -r ./mongolian-bert/requirements.txt

    100% |████████████████████████████████| 1.0MB 15.2MB/s 
    100% |████████████████████████████████| 81kB 23.9MB/s 
    100% |████████████████████████████████| 112kB 34.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/84/11/01/951369cbbf8f96878786a1f4da68bd7ac19a5d945b38e03d54
Successfully built EbookLib


In [5]:
from tokenization_sentencepiece import FullTokenizer
model_path = join(project_path, "sentencepiece")
tokenizer  = FullTokenizer(model_file=join(model_path, 'mn_cased.model'), vocab_file=join(model_path, 'mn_cased.vocab'), do_lower_case=False)

Loaded a trained SentencePiece model.


In [6]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.109.213.42:8470

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 6815742135551305842),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 3977030200985302364),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 10136270177372746879),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 13836083277283955847),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 1377376880298993897),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 754998688830016202),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 171

In [7]:
BERT_PRETRAINED_DIR = MODEL_BUCKET
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
!gsutil ls $BERT_PRETRAINED_DIR

***** BERT pretrained directory: gs://mongolian-bert-models/model-32k-512-4000000 *****
gs://mongolian-bert-models/model-32k-512-4000000/checkpoint
gs://mongolian-bert-models/model-32k-512-4000000/eval_results.txt
gs://mongolian-bert-models/model-32k-512-4000000/events.out.tfevents.1553378588.mongolian-bert
gs://mongolian-bert-models/model-32k-512-4000000/events.out.tfevents.1553515553.mongolian-bert
gs://mongolian-bert-models/model-32k-512-4000000/events.out.tfevents.1553680754.mongolian-bert
gs://mongolian-bert-models/model-32k-512-4000000/events.out.tfevents.1553890194.mongolian-bert
gs://mongolian-bert-models/model-32k-512-4000000/events.out.tfevents.1553966034.mongolian-bert
gs://mongolian-bert-models/model-32k-512-4000000/events.out.tfevents.1554236264.mongolian-bert
gs://mongolian-bert-models/model-32k-512-4000000/graph.pbtxt
gs://mongolian-bert-models/model-32k-512-4000000/model.ckpt-4000000.data-00000-of-00001
gs://mongolian-bert-models/model-32k-512-4000000/model.ckpt-4000000

In [8]:
BUCKET  = 'mongolian-bert-models'
DATASET = 'eduge'
LEN     = 512
OUTPUT_DIR = 'gs://{}/{}/{}_cased'.format(BUCKET, DATASET, LEN)
print(OUTPUT_DIR)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

gs://mongolian-bert-models/eduge/512_cased
***** Model output directory: gs://mongolian-bert-models/eduge/512_cased *****


In [0]:
sys.path.append("./mongolian-bert/bert")
import modeling
import optimization
import run_classifier
import tokenization

In [0]:
VOCAB_FILE      = os.path.join("./mongolian-bert/{}".format(MODEL), 'mn_cased.vocab')
CONFIG_FILE     = os.path.join("./mongolian-bert/{}".format(MODEL), 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, MODEL_CHECKPOINT)
DO_LOWER_CASE   = False

In [11]:
import os
if not os.path.exists("eduge.csv.gz"):
  !wget https://github.com/tugstugi/mongolian-nlp/raw/master/datasets/eduge.csv.gz
  !gunzip eduge.csv.gz

--2019-04-07 17:46:23--  https://github.com/tugstugi/mongolian-nlp/raw/master/datasets/eduge.csv.gz
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tugstugi/mongolian-nlp/master/datasets/eduge.csv.gz [following]
--2019-04-07 17:46:23--  https://raw.githubusercontent.com/tugstugi/mongolian-nlp/master/datasets/eduge.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73951267 (71M) [application/octet-stream]
Saving to: ‘eduge.csv.gz’

eduge.csv.gz        100%[===================>]  70.53M   152MB/s    in 0.5s    

2019-04-07 17:46:25 (152 MB/s) - ‘eduge.csv.gz’ saved [73951267/73

In [12]:
import pandas as pd
df = pd.read_csv("eduge.csv")
df = df.rename(columns=lambda x: x.strip())
df.columns.values
labels = df['label'].unique().tolist()
labels


['урлаг соёл',
 'эдийн засаг',
 'эрүүл мэнд',
 'хууль',
 'улс төр',
 'спорт',
 'технологи',
 'боловсрол',
 'байгал орчин']

In [0]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.1, random_state=999, stratify=df['label'])

In [0]:
DATA_COLUMN  = 'news'
LABEL_COLUMN = 'label'
label_list   = labels

In [0]:
train_InputExamples = train.apply(lambda x: run_classifier.InputExample(guid=None,
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label  = x[LABEL_COLUMN]), axis = 1)
test_InputExamples  = test.apply(lambda x: run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label  = x[LABEL_COLUMN]), axis = 1)

In [0]:
LEARNING_RATE          = 2e-5
NUM_TRAIN_EPOCHS       = 20.0
WARMUP_PROPORTION      = 0.1
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS     = 100
TRAIN_BATCH_SIZE       = 128 #256
EVAL_BATCH_SIZE        = 128 #256
PREDICT_BATCH_SIZE     = 8
ITERATIONS_PER_LOOP    = 1000
NUM_TPU_CORES          = 8
MAX_SEQ_LENGTH         = LEN

should_train           = True

In [17]:
train_features = run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features  = run_classifier.convert_examples_to_features(test_InputExamples , label_list, MAX_SEQ_LENGTH, tokenizer)

num_train_steps  = int(len(train_features) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_test_steps   = int(len(test_features) / EVAL_BATCH_SIZE)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

INFO:tensorflow:Writing example 0 of 68094
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: None
INFO:tensorflow:tokens: [CLS] ▁Засгийн ▁газрын ▁гишүүн , ▁УИХ - ын ▁гишүүн ▁Х . Баттулга ▁сайд аасаа ▁тат зал зах ▁хүсэлтээ ▁өнгөрсөн ▁долоо ▁хоногт ▁өгсөн ▁билээ . ▁Тэрээр ▁сайд аасаа ▁татгалзах ▁хүсэлт дээ , ▁Засгийн ▁газрын ▁гишүүний ▁ажлаа ▁өгч , ▁УИХ - ын ▁гишүүний ▁үүргээ ▁үргэлж үүлэх ээр ▁болсноо ▁дурьдсан ▁бөгөөд ▁цаашид ▁үйлдвэрлэл , ▁хөдөө ▁аж ▁ахуйн ▁салбарт ▁эхлүүлсэн ▁ажлаа ▁үргэлж үүлэхэд ▁анхаарч ▁ажиллана ▁гэдгээ ▁мэдэгдсэн . ▁Өнгөрсөн ▁долоо ▁хоногт ▁Ерөнхийлөгчөөс ▁өргөн ▁барьсан ▁Засгийн ▁газрын ▁гишүүний ▁үүргийг ▁давхар ▁хашдаг ▁УИХ - ын ▁гишүүдийг ▁" дан ▁дээлтэй " ▁болгох ▁хуулийн ▁төслийг ▁гурван ▁сарын ▁хугацаанд ▁хэлэлцэж , ▁унагасан . ▁УИХ - ын ▁гишүүдийг ▁" давхар ▁дээл "- ийг ▁тайлах ▁хуулийн ▁төслийг ▁2014 ▁оноос ▁хэрэгжүүлэх ▁ёстой ▁гэдэг ▁дээр ▁Х . Баттулга ▁сайд ▁тууштай ▁байсан ▁бөгөөд ▁өгсөн ▁саналаа ▁бататгаж , ▁өргөдлөө ▁өгсөн ▁гэж ▁зарим ▁хүмүүс ▁

In [18]:
CONFIG_FILE = "./mongolian-bert/bert_configs/bert_base_config.json"
print(CONFIG_FILE)

./mongolian-bert/bert_configs/bert_base_config.json


In [19]:
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
run_config = tf.contrib.tpu.RunConfig(
    cluster   = tpu_cluster_resolver,
    model_dir = OUTPUT_DIR,
    save_checkpoints_steps = SAVE_CHECKPOINTS_STEPS,
    tpu_config = tf.contrib.tpu.TPUConfig(
        iterations_per_loop         = ITERATIONS_PER_LOOP,
        num_shards                  = NUM_TPU_CORES,
        per_host_input_for_training = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

model_fn = run_classifier.model_fn_builder(
    bert_config      = modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels       = len(label_list),
    init_checkpoint  = INIT_CHECKPOINT,
    learning_rate    = LEARNING_RATE,
    num_train_steps  = num_train_steps,
    num_warmup_steps = num_warmup_steps,
    use_tpu          = True,
    use_one_hot_embeddings = True)

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu            = True,
    model_fn           = model_fn,
    config             = run_config,
    train_batch_size   = TRAIN_BATCH_SIZE,
    eval_batch_size    = EVAL_BATCH_SIZE,
    predict_batch_size = PREDICT_BATCH_SIZE)


INFO:tensorflow:Using config: {'_model_dir': 'gs://mongolian-bert-models/eduge/512_cased', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.109.213.42:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0736faecc0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.109.213.42:8470', '_evaluation_master': 'grpc://10.109.213.42:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, nu

In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = run_classifier.input_fn_builder(
    features       = train_features,
    seq_length     = MAX_SEQ_LENGTH,
    is_training    = True,
    drop_remainder = True)

In [21]:
from datetime import datetime

if should_train:
  print("Beginning Training!")
  current_time = datetime.now()
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Querying Tensorflow master (grpc://10.109.213.42:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 6815742135551305842)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 3977030200985302364)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 10136270177372746879)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 13836083277283955847)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 1377376880298993897)
INFO:tensorflow:*** Available Devic

In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features       = test_features,
    seq_length     = MAX_SEQ_LENGTH,
    is_training    = False,
    drop_remainder = True)

In [23]:
estimator.evaluate(input_fn = test_input_fn, steps=num_test_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (16, 512)
INFO:tensorflow:  name = input_mask, shape = (16, 512)
INFO:tensorflow:  name = label_ids, shape = (16,)
INFO:tensorflow:  name = segment_ids, shape = (16, 512)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (32000, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/bias:

{'eval_accuracy': 0.9165784,
 'eval_loss': 0.64956117,
 'global_step': 10639,
 'loss': 0.7359641}

In [0]:
def getPrediction(in_sentences):
  input_examples   = [run_classifier.InputExample(guid=None, text_a = x, text_b = None, label = "урлаг соёл") for x in in_sentences]
  input_features   = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
  predictions      = estimator.predict(predict_input_fn)
  return predictions

In [0]:
pred_sentences = [
  "Монгол улсын уул уурхайн гарц энэ жилээс эхлэн нэмэгдэж дэлхийн эдийн засаг сэргэсний азаар эдийн засагт таатай сайхан нөхцөл байдал үүсээд байна.",
  "Хиймэл оюун болоод дуу хоолой таних систем ухаалаг туслагчийн үр нөлөөгөөр ухаалаг утас хэрэглэгчдийн тоо эрс нэмэгджээ.",
  "Гурвыг харьцах гурав заалны сагсан бөмбөгийн спортод манай тамирчид одтой байна.",
  "Ханиад томуунаас урьдчилан сэргийлэхийн тулд биеэ чийрэгжүүлэх шаардлагатай."    
]

In [0]:
import numpy as np

In [27]:
predictions = getPrediction(pred_sentences)
for (sent, pred) in zip(pred_sentences, predictions):
  print("==============SENTENCE===============")
  print(sent)
  probabilities = pred["probabilities"]
  softmax       = np.exp(probabilities)/sum(np.exp(probabilities))
  label_index   = np.argmax(softmax)
  #print(probabilities)
  #print(softmax)
  #print(label_index)
  print(label_list[label_index])

INFO:tensorflow:Writing example 0 of 4
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: None
INFO:tensorflow:tokens: [CLS] ▁Монгол ▁улсын ▁уул ▁уурхайн ▁гарц ▁энэ ▁жилээс ▁эхлэн ▁нэмэгдэж ▁дэлхийн ▁эдийн ▁засаг ▁сэргэ сний ▁азаар ▁эдийн ▁засагт ▁таатай ▁сайхан ▁нөхцөл ▁байдал ▁үүсээд ▁байна . [SEP]
INFO:tensorflow:input_ids: 4 29 44 580 682 1911 22 2643 831 1219 272 88 335 11703 2339 21142 88 1550 1300 278 368 205 4346 11 7 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0